In [1]:
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
from datetime import datetime, timedelta

# Test "Buy on new moon, sell on full moon"
Claim: "Buy on new moon, sell on full moon =  positive expectation over many years....however, buy on full moon, sell on new moon = losing strategy - so serves as a control against the upward bias to the market."
https://twitter.com/LindaRaschke/status/1740066413132275885

## Get moon phase dates for the past 30 years
### I used https://aa.usno.navy.mil/data/MoonPhases as the source
### From this site, I created csv files with the month, year, and day components of each phase
### The phase data is based on UTC

In [2]:
# Adjust as needed for your environment
FILEPATH = "c:/Users/steve/Downloads/"

In [3]:
# Read new moon csv file
filename = FILEPATH + "new_moon_date_components.csv"
new_moon_dates = pd.read_csv(filename)
# Set DateTimeIndex on new moon dates
new_moon_dates['Date'] = pd.to_datetime(new_moon_dates).dt.tz_localize("US/Eastern")
new_moon_dates.set_index(new_moon_dates['Date'], inplace=True)
# Designate that these are new moon dates
new_moon_dates['phase'] = 'new'

In [4]:
new_moon_dates

,year,month,day,Date,phase
Date,,,,,
1993-12-13 00:00:00-05:00,1993,12,13,1993-12-13 00:00:00-05:00,new
1994-01-11 00:00:00-05:00,1994,1,11,1994-01-11 00:00:00-05:00,new
1994-02-10 00:00:00-05:00,1994,2,10,1994-02-10 00:00:00-05:00,new
1994-03-12 00:00:00-05:00,1994,3,12,1994-03-12 00:00:00-05:00,new
1994-04-11 00:00:00-04:00,1994,4,11,1994-04-11 00:00:00-04:00,new
...,...,...,...,...,...
2023-08-16 00:00:00-04:00,2023,8,16,2023-08-16 00:00:00-04:00,new
2023-09-15 00:00:00-04:00,2023,9,15,2023-09-15 00:00:00-04:00,new
2023-10-14 00:00:00-04:00,2023,10,14,2023-10-14 00:00:00-04:00,new


In [5]:
# Repeat the above steps for full moon dates
filename = FILEPATH + "full_moon_date_components.csv"
full_moon_dates = pd.read_csv(filename)
# Set DateTimeIndex on full moon dates
full_moon_dates['Date'] = pd.to_datetime(full_moon_dates).dt.tz_localize("US/Eastern")
full_moon_dates.set_index(full_moon_dates['Date'], inplace=True)
# Designate that these are full moon dates
full_moon_dates['phase'] = 'full'

In [6]:
full_moon_dates

,year,month,day,Date,phase
Date,,,,,
1993-12-28 00:00:00-05:00,1993,12,28,1993-12-28 00:00:00-05:00,full
1994-01-27 00:00:00-05:00,1994,1,27,1994-01-27 00:00:00-05:00,full
1994-02-26 00:00:00-05:00,1994,2,26,1994-02-26 00:00:00-05:00,full
1994-03-27 00:00:00-05:00,1994,3,27,1994-03-27 00:00:00-05:00,full
1994-04-25 00:00:00-04:00,1994,4,25,1994-04-25 00:00:00-04:00,full
...,...,...,...,...,...
2023-08-31 00:00:00-04:00,2023,8,31,2023-08-31 00:00:00-04:00,full
2023-09-29 00:00:00-04:00,2023,9,29,2023-09-29 00:00:00-04:00,full
2023-10-28 00:00:00-04:00,2023,10,28,2023-10-28 00:00:00-04:00,full


In [7]:
# Concatenate the new and full moon data into a single dataframe
moon_phase_dates = pd.concat([new_moon_dates, full_moon_dates])
#  Move the timestamps 2 hours earlier so they will sort ahead of the stock market trading days
moon_phase_dates.index = moon_phase_dates.index.shift(-2, freq='H')

## Get daily SPY prices

In [8]:
# Get S&P 500 daily prices
spy = yf.Ticker('SPY')
spy_df = spy.history(start="1993-12-06", end="2023-12-29", interval="1d")

In [9]:
spy_df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
1993-12-06 00:00:00-05:00,26.928421,27.018362,26.928421,26.982386,99500,0.0,0.0,0.0
1993-12-07 00:00:00-05:00,26.982376,27.000364,26.928411,26.964388,88800,0.0,0.0,0.0
1993-12-08 00:00:00-05:00,26.964388,26.964388,26.928411,26.964388,146700,0.0,0.0,0.0
1993-12-09 00:00:00-05:00,26.964405,27.000382,26.838488,26.874464,416500,0.0,0.0,0.0
1993-12-10 00:00:00-05:00,26.892445,26.892445,26.766527,26.820492,412900,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2023-12-21 00:00:00-05:00,471.329987,472.980011,468.839996,472.700012,86667500,0.0,0.0,0.0
2023-12-22 00:00:00-05:00,473.859985,475.380005,471.700012,473.649994,67126600,0.0,0.0,0.0
2023-12-26 00:00:00-05:00,474.070007,476.579987,473.989990,475.649994,55387000,0.0,0.0,0.0


## Sort the SPY daily prices with the moon phases

In [10]:
# Concatenate the new and full moon data into a single dataframe
combined = pd.concat([spy_df, moon_phase_dates])
# Sort the combined data so that the SPY prices and moon phases are in the same sequence
combined.sort_index(inplace=True)

In [11]:
combined.head(10)

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains,year,month,day,Date,phase
Date,,,,,,,,,,,,,
1993-12-06 00:00:00-05:00,26.928421,27.018362,26.928421,26.982386,99500.0,0.0,0.0,0.0,NaN,NaN,NaN,NaT,NaN
1993-12-07 00:00:00-05:00,26.982376,27.000364,26.928411,26.964388,88800.0,0.0,0.0,0.0,NaN,NaN,NaN,NaT,NaN
1993-12-08 00:00:00-05:00,26.964388,26.964388,26.928411,26.964388,146700.0,0.0,0.0,0.0,NaN,NaN,NaN,NaT,NaN
1993-12-09 00:00:00-05:00,26.964405,27.000382,26.838488,26.874464,416500.0,0.0,0.0,0.0,NaN,NaN,NaN,NaT,NaN
1993-12-10 00:00:00-05:00,26.892445,26.892445,26.766527,26.820492,412900.0,0.0,0.0,0.0,NaN,NaN,NaN,NaT,NaN
1993-12-12 22:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1993.0,12.0,13.0,1993-12-13 00:00:00-05:00,new
1993-12-13 00:00:00-05:00,26.820491,26.982386,26.784515,26.982386,273200.0,0.0,0.0,0.0,NaN,NaN,NaN,NaT,NaN
1993-12-14 00:00:00-05:00,27.000365,27.000365,26.784506,26.784506,41900.0,0.0,0.0,0.0,NaN,NaN,NaN,NaT,NaN
1993-12-15 00:00:00-05:00,26.820494,26.838482,26.748541,26.748541,82600.0,0.0,0.0,0.0,NaN,NaN,NaN,NaT,NaN


## Test the strategy: Buy at new moon, sell at full moon
Because moon phases might occur on weekends, take action at the close of the last trading day before each phase

In [12]:
combined['sell_setup'] = (combined['phase'].shift(-1) == 'full')
combined['buy_setup'] = (combined['phase'].shift(-1) == 'new')

In [13]:
combined.head(20)

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains,year,month,day,Date,phase,sell_setup,buy_setup
Date,,,,,,,,,,,,,,,
1993-12-06 00:00:00-05:00,26.928421,27.018362,26.928421,26.982386,99500.0,0.000,0.0,0.0,NaN,NaN,NaN,NaT,NaN,False,False
1993-12-07 00:00:00-05:00,26.982376,27.000364,26.928411,26.964388,88800.0,0.000,0.0,0.0,NaN,NaN,NaN,NaT,NaN,False,False
1993-12-08 00:00:00-05:00,26.964388,26.964388,26.928411,26.964388,146700.0,0.000,0.0,0.0,NaN,NaN,NaN,NaT,NaN,False,False
1993-12-09 00:00:00-05:00,26.964405,27.000382,26.838488,26.874464,416500.0,0.000,0.0,0.0,NaN,NaN,NaN,NaT,NaN,False,False
1993-12-10 00:00:00-05:00,26.892445,26.892445,26.766527,26.820492,412900.0,0.000,0.0,0.0,NaN,NaN,NaN,NaT,NaN,False,True
1993-12-12 22:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1993.0,12.0,13.0,1993-12-13 00:00:00-05:00,new,False,False
1993-12-13 00:00:00-05:00,26.820491,26.982386,26.784515,26.982386,273200.0,0.000,0.0,0.0,NaN,NaN,NaN,NaT,NaN,False,False
1993-12-14 00:00:00-05:00,27.000365,27.000365,26.784506,26.784506,41900.0,0.000,0.0,0.0,NaN,NaN,NaN,NaT,NaN,False,False
1993-12-15 00:00:00-05:00,26.820494,26.838482,26.748541,26.748541,82600.0,0.000,0.0,0.0,NaN,NaN,NaN,NaT,NaN,False,False


In [14]:
# Select the days with buy or sell setups
transactions = combined.loc[(combined['buy_setup'] == True) | (combined['sell_setup'] == True)][['Close', 'buy_setup', 'sell_setup']]

In [15]:
transactions

,Close,buy_setup,sell_setup
Date,,,
1993-12-10 00:00:00-05:00,26.820492,True,False
1993-12-27 00:00:00-05:00,27.239521,False,True
1994-01-10 00:00:00-05:00,27.583664,True,False
1994-01-26 00:00:00-05:00,27.420650,False,True
1994-02-09 00:00:00-05:00,27.474993,True,False
...,...,...,...
2023-10-27 00:00:00-04:00,409.021637,False,True
2023-11-10 00:00:00-05:00,438.830780,True,False
2023-11-24 00:00:00-05:00,453.461456,False,True


## Calculate net changes of entire time period (population) and of trades (sample)

In [16]:
# To avoid SettingWithCopyWarning issues, write the calculation results out as series.
log_return = np.log(transactions['Close']) - np.log(transactions['Close'].shift(1))
log_return.rename('log_return', inplace=True)

Date
1993-12-10 00:00:00-05:00         NaN
1993-12-27 00:00:00-05:00    0.015503
1994-01-10 00:00:00-05:00    0.012555
1994-01-26 00:00:00-05:00   -0.005927
1994-02-09 00:00:00-05:00    0.001980
                               ...   
2023-10-27 00:00:00-04:00   -0.049453
2023-11-10 00:00:00-05:00    0.070346
2023-11-24 00:00:00-05:00    0.032796
2023-12-11 00:00:00-05:00    0.014587
2023-12-26 00:00:00-05:00    0.033185
Name: log_return, Length: 744, dtype: float64

In [17]:
population = pd.concat([transactions, log_return], axis="columns")

In [18]:
population

,Close,buy_setup,sell_setup,log_return
Date,,,,
1993-12-10 00:00:00-05:00,26.820492,True,False,NaN
1993-12-27 00:00:00-05:00,27.239521,False,True,0.015503
1994-01-10 00:00:00-05:00,27.583664,True,False,0.012555
1994-01-26 00:00:00-05:00,27.420650,False,True,-0.005927
1994-02-09 00:00:00-05:00,27.474993,True,False,0.001980
...,...,...,...,...
2023-10-27 00:00:00-04:00,409.021637,False,True,-0.049453
2023-11-10 00:00:00-05:00,438.830780,True,False,0.070346
2023-11-24 00:00:00-05:00,453.461456,False,True,0.032796


In [19]:
sample = population.loc[population['sell_setup'] == True]

In [20]:
sample

,Close,buy_setup,sell_setup,log_return
Date,,,,
1993-12-27 00:00:00-05:00,27.239521,False,True,0.015503
1994-01-26 00:00:00-05:00,27.420650,False,True,-0.005927
1994-02-25 00:00:00-05:00,27.130878,False,True,-0.012604
1994-03-25 00:00:00-05:00,26.777332,False,True,-0.013784
1994-04-22 00:00:00-04:00,26.148878,False,True,0.003839
...,...,...,...,...
2023-08-30 00:00:00-04:00,447.609924,False,True,0.018168
2023-09-28 00:00:00-04:00,426.789612,False,True,-0.046189
2023-10-27 00:00:00-04:00,409.021637,False,True,-0.049453


## Calculate backtest statistics

In [21]:
sample['log_return'].describe()

count    372.000000
mean       0.002553
std        0.033725
min       -0.178771
25%       -0.011664
50%        0.006271
75%        0.021402
max        0.173273
Name: log_return, dtype: float64

In [22]:
print("The sample mean log return is ", sample['log_return'].mean())
print("The population mean log return is ", population['log_return'].mean())

The sample mean log return is  0.0025530119074633284
The population mean log return is  0.0038701427488792707


In [23]:
print("Backtest sample statistics (log returns):")
print("Mean: ", sample['log_return'].mean())
print("Standard deviation: ", sample['log_return'].std())
print("N: ", sample['log_return'].count())
print("t: ", (sample['log_return'].mean() - population['log_return'].mean()) / (sample['log_return'].std() / (sample['log_return'].count() ** 0.5)))

Backtest sample statistics (log returns):
Mean:  0.0025530119074633284
Standard deviation:  0.0337245207472958
N:  372
t:  -0.7532768181541406
